# Aprendizaje supervisado

## 1. Introducción

Los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Es necesario predecir si un cliente dejará el banco pronto.

Se creará un modelo con el máximo valor F1 posible. Para aprobar la revisión, necesitas un valor F1 de al menos 0.59. Se verifica F1 para el conjunto de prueba. 

Además, se debe medir la métrica AUC-ROC y compararla con el valor F1.

## 2. Preparación de los datos

Se descargarán los datos desde el archivo csv y se buscarán valores repetidos o ausentes. También se revisrá el tipo de datos de cada columna

In [5]:
# importación de librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle


In [6]:
# Descarga de los datos
data = pd.read_csv('/datasets/Churn.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [7]:
# Se imprime una muestra de los datos y se buscan datos duplicados
display(data.sample(5))
print()
print('Cantidad de filas duplicadas:', data.duplicated().sum())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9132,9133,15737194,Tu,635,France,Female,33,5.0,0.00,2,1,0,122949.71,0
9789,9790,15814040,Munroe,610,France,Female,45,1.0,0.00,2,1,1,199657.46,0
3120,3121,15694879,Reeves,590,Spain,Female,23,7.0,0.00,2,1,0,196789.90,0
776,777,15712551,Shen,622,Germany,Female,58,7.0,116922.25,1,1,0,120415.61,1
3736,3737,15607748,Bennett,498,Germany,Male,37,8.0,108432.88,2,1,1,14865.05,0



Cantidad de filas duplicadas: 0


Conclusiones:

* La columna `'Tenure'` tiene 909 valores ausentes, los cuales representan un 9% del total de filas de los datos. La librería `scikit-learn` es incompatible para trabajar con datos ausentes. La cantidad de datos es considerable para que sean remplazados por valores constantes (como la media), esto creará un gran sesgo en los datos. Con esto se decide eliminar las filas que tienen valores ausentes en la columna `'Tenure'`.
* La columna `'Surname'` será eliminada debido a la gran cantidad de apellidos que hay y al momento de realizar la codificación OHE habrán muchas columnas y los cálculos con la computadora tomarán mucho tiempo, además los apellidos no son de importancia cuando un usuario decide dejar el banco. Adicionalmente con la columna `'CustomerId'` se puede saber que usuario es.
* También se ha decidido borrar las columnas `RowNumber` y `CustomerId`. Ya que son el índice de cadena de datos y el identificador de cliente único, respectivamente. Ninguno afecta al momento de que un cliente decide dejar la sucursal
* Los tipos de datos están correctos
* No hay datos duplicados

In [8]:
# Borrar filas con valores ausentes
data = data.dropna()

# Borra la columnas
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      9091 non-null   int64  
 1   Geography        9091 non-null   object 
 2   Gender           9091 non-null   object 
 3   Age              9091 non-null   int64  
 4   Tenure           9091 non-null   float64
 5   Balance          9091 non-null   float64
 6   NumOfProducts    9091 non-null   int64  
 7   HasCrCard        9091 non-null   int64  
 8   IsActiveMember   9091 non-null   int64  
 9   EstimatedSalary  9091 non-null   float64
 10  Exited           9091 non-null   int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 852.3+ KB


### 2.1. Codificación de características categóricas utilizando OHE y escalado de las características numéricas.

Se transformarán las características categóricas en numéricas para utilizar modelos de regresión, evitando la trampa dummy. Después de esto, se dividirá el conjunto en las caracteríscticas y el objetivo.

In [9]:
# Codificación OHE
data_ohe = pd.get_dummies(data, drop_first=True)

# División de características y objetivo
target = data_ohe['Exited']
features = data_ohe.drop('Exited', axis=1)

Se hará la división de datos en los conjuntos de entrenamiento, validación y prueba. Para lograrlo se hará uso de `train_test_split` dos veces y así, poder separar el dataset en tres.

In [10]:
# División de los conjuntos de etrenamiento y prueba
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.2, random_state=12345)

# División de entrenamiento y validación
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size = 0.25, random_state=12345)
# # Se usa 0.25 porque 0.8*0.25=0.2

A continuación se realizará el escalado de características y se volverá a dividir el conjunto de características en entrenamiento y validación, para que el modelo considere todas las características igual de importantes.

In [11]:
# Primero sacaremos las columnas que que son numéricas desde el inicio
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
           'EstimatedSalary']

# Escalado de características
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])

# Comprobamos las filas y columnas
print(features_train.shape)
print(features_valid.shape)

(5454, 11)
(1818, 11)


## 3. Entrenamiento de modelos sin tomar en cuenta el desequilibrio

Se entrenaran 3 modelos: Árbol de decisión, Bosque aleatorio y Regresión logística. A cada modelo se le obtendrá el valor F1 para tratar de aproximarnos al valor solicitado.

También se ha solicitado que obtengamos el valor AUC-ROC de cada modelo y compararlos, por lo que también se calculará este valor

### 3.1. Árbol de regresión

Se va a entrenar el modelo en diferentes profundidades de árbol para saber cuál es la profundidad que tiene el mayor valor F1

In [12]:
# Variables para guardar la mejor f1 con el max_depth
best_f1_tree = 0
best_depth = 0

# Ciclo para evaluar el modelo en diferentes profundidades
for depth in range(1,11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions_valid_tree = model_tree.predict(features_valid)
    f1 = f1_score(target_valid, predictions_valid_tree)
    if f1 > best_f1_tree:
        best_depth = depth
        best_f1_tree = f1

print(f'F1 del mejor modelo en el conjunto de validación (max_depth={best_depth}): {best_f1_tree}')

F1 del mejor modelo en el conjunto de validación (max_depth=9): 0.557427258805513


Valor AUC-ROC.

Para poder sacar este valor es necesario tomar las probabilidades de clase positiva y nos muestra el área bajo la curva de la característica operativa del receptor.

In [13]:
# Probabilidades de clase
probabilities_valid = model_tree.predict_proba(features_valid)

# Probabilidades de clase positiva
probabilities_one_valid = probabilities_valid[:,1]

# valor AUC-ROC
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC_ROC:', auc_roc)

AUC_ROC: 0.7799276294991645


Conclusión:

La mejor profundidad de árbol es de 7, cuyo valor es de 0.56, sin embargo, se nos solicitó que por lo menos tuviera un valor de 0.59. Este modelo queda descartado.

El valor AUC-ROC es más grande que el F1 pero aún está lejos del 1

### 3.2. Bosque aleatorio

Se va a entrenar el modelo en difirentes valores de `n_estimators` para saber cuál tiene mejor valor F1. Por motivos de cómputo, sólo se evaluarán hasta 10 árboles

In [14]:
# Variables para guardar la mejor f1 con el número de de árboles
best_f1_forest = 0
best_estimator = 0

# Ciclo para evaluar el modelo en diferentes números de árboles
for est in range(1,11):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_forest.fit(features_train, target_train)
    predictions_valid_forest = model_forest.predict(features_valid)
    f1 = f1_score(target_valid, predictions_valid_forest)
    if f1 > best_f1_forest:
        best_estimator = est
        best_f1_forest = f1

print(f'F1 del mejor modelo en el conjunto de validación (n_estimators={best_estimator}): {best_f1_forest}')

F1 del mejor modelo en el conjunto de validación (n_estimators=7): 0.523961661341853


Valor AUC-ROC.

In [15]:
# Probabilidades de clase
probabilities_valid = model_forest.predict_proba(features_valid)

# Probabilidades de clase positiva
probabilities_one_valid = probabilities_valid[:,1]

# valor AUC-ROC
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC_ROC:', auc_roc)

AUC_ROC: 0.7948589847807433


Conclusión:

El mejor número de árbol es 8, cuyo valor es de 0.50, sin embargo, se nos solicitó que por lo menos tuviera un valor de 0.59. Este modelo queda descartado.

El valor AUC-ROC es más grande que el F1 pero aún está lejos del 1

### 3.3. Regresión Logística

Se entrena el modelo con `solver=liblinear`

In [16]:
model_regression = LogisticRegression(random_state=12345, solver='liblinear')
model_regression.fit(features_train, target_train)
predictions_valid_regression = model_regression.predict(features_valid)
f1_regression = f1_score(target_valid, predictions_valid_regression)
print('Valor F1:', f1_regression)

Valor F1: 0.3004115226337448


Valor AUC-ROC.

In [17]:
# Probabilidades de clase
probabilities_valid = model_regression.predict_proba(features_valid)

# Probabilidades de clase positiva
probabilities_one_valid = probabilities_valid[:,1]

# valor AUC-ROC
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC_ROC:', auc_roc)

AUC_ROC: 0.7725917897303888


Conclusión:

El valor F1 del modelo es de 0.30, sin embargo, se nos solicitó que por lo menos tuviera un valor de 0.59. Este modelo queda descartado.

El valor AUC-ROC es más grande que el F1 pero aún está lejos del 1

### 3.4. Conclusiones

Los modelos con desequilibrio de clases no alcanzaron el valor F1 que fue solicitado en el ejercicio. Ahora se harán los modelos austando ese desbalanceo.

El valor F1 es muy variado entre cada modelo. Por otro lado, el valor AUC-ROC no cambia entre cada modelo.

## 4. Mejora de la calidad de los modelos

Para lograr esto se harán dos enfoques para corregir el desequilibrio de clase: Sobremuestreo y submuestreo.

En el sobremuestreo las tareas más importantes se repiten, mientras que en el submuestreo hcemos que las observaciones de clase más frecuentes sean menos frecuentes. Dicho esto, necesitamos averiguar que clase se repite más veces.

Cuando sepamos cuál clase se repite más, se aplicarán los métodos antes mencionados para hacer que las clases estén más quelibradas. De cada método se obtendrán características y objetivos con los que se evaluará el modelo.

### 4.1. ¿Cuál clase se repite más?

Para saber cuál es la clase que más se repite se dividirán los datos de entrenamiento en observaciones negativas y positivas. Es decir, se realizarán filtros de las características y objetivo de entrenamiento donde el objetivo sea 0 y uno.

In [18]:
# División del conjunto de datos
features_zeros = features_train[target_train==0]
features_ones = features_train[target_train==1]
target_zeros = target_train[target_train==0]
target_ones = target_train[target_train==1]
print(features_zeros.shape)
print(features_ones.shape)
print(target_zeros.shape)
print(target_ones.shape)

(4335, 11)
(1119, 11)
(4335,)
(1119,)


Conclusión:

Hay alrededor de 4 veces más observaciones negativas que positivas

### 4.2. Sobremuestreo

De acuerdo con el ejercicio anterior, tenemos que duplicar las observaciones positivas. Para lograrlo se hace uso de la función `concat()` de la librería pandas. Se necesitará un multiplicador o repetidor para aumentar las observaciones positivas.

In [19]:
# Número de repeticiones
repeat = 9

# Características y objetivo aumentadas usando concat()
features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
print(features_upsampled.shape)
print(target_upsampled.shape)

(14406, 11)
(14406,)


Ahora es necesario "barajear" los datos, para esto hacemos uso de la función `shuffle()`

In [20]:
# Función shuffle()
features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

### 4.3. Submuestreo

Para esto haremos uso de los resultados del punto 4.1. Pero a diferencia del sobremuestreo, ahora se descartarán al azar una parte de las observaciones negativas haceindo uso de la función `sample()` con el parámetro `fracc`, porque tenemos que determinar que fracción de los datos queremos soltar.

In [21]:
# Fracción a soltar
fraction = 0.4

# Descartar observaciones negativas con sample()
features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])

print(features_downsampled.shape)
print(target_downsampled.shape)

(2853, 11)
(2853,)


De igual manera que con el sobremuestreo, se tienen que barajear los datos

In [22]:
# Función shuffle()
features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)

Con los datos equilibrados se volverán a entrenar los modelos de árboles de desición, bosque aleatorio y Regresión logística en cada conjunto: sobremuestreo y submuestreo

### 4.4. Árbol de decisión

Se va a entrenar el modelo en diferentes profundidades de árbol para saber cuál es la profundidad que tiene el mayor valor F1. También se obtendrá el valor AUC-ROC

#### 4.4.1. Conjunto con sobremuestreo

In [23]:
# Variables para guardar la mejor f1 con el max_depth
best_f1_tree_up = 0
best_depth = 0

# Ciclo para evaluar el modelo en diferentes profundidades
for depth in range(1,11):
    model_tree_up = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree_up.fit(features_upsampled, target_upsampled)
    predictions_valid = model_tree_up.predict(features_valid)
    f1 = f1_score(target_valid, predictions_valid)
    if f1 > best_f1_tree_up:
        best_depth = depth
        best_f1_tree_up = f1

print(f'F1 del mejor modelo en el conjunto de validación (max_depth={best_depth}): {best_f1_tree_up}')

F1 del mejor modelo en el conjunto de validación (max_depth=9): 0.5048715677590788


Valor AUC-ROC

In [24]:
# Probabilidades de clase
probabilities_valid = model_tree_up.predict_proba(features_valid)

# Probabilidades de clase positiva
probabilities_one_valid = probabilities_valid[:,1]

# valor AUC-ROC
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC_ROC:', auc_roc)

AUC_ROC: 0.7625726339399961


#### 4.4.2. Conjunto con submuestreo

In [25]:
# Variables para guardar la mejor f1 con el max_depth
best_f1_tree_down = 0
best_depth = 0

# Ciclo para evaluar el modelo en diferentes profundidades
for depth in range(1,11):
    model_tree_down = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree_down.fit(features_downsampled, target_downsampled)
    predictions_valid = model_tree_down.predict(features_valid)
    f1 = f1_score(target_valid, predictions_valid)
    if f1 > best_f1_tree_down:
        best_depth = depth
        best_f1_tree_down = f1

print(f'F1 del mejor modelo en el conjunto de validación (max_depth={best_depth}): {best_f1_tree_down}')

F1 del mejor modelo en el conjunto de validación (max_depth=5): 0.577127659574468


Valor AUC-ROC

In [26]:
# Probabilidades de clase
probabilities_valid = model_tree_down.predict_proba(features_valid)

# Probabilidades de clase positiva
probabilities_one_valid = probabilities_valid[:,1]

# valor AUC-ROC
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC_ROC:', auc_roc)

AUC_ROC: 0.7374179575185535


Conclusiones:

No se logra el valor F1 deseado, de es más bajo con ambos conjuntos mejorados, de igual manera el valor AUC-ROC es más bajo en los dos conuntos

### 4.5. Bosque Aleatorio

Se va a entrenar el modelo en difirentes valores de `n_estimators` para saber cuál tiene mejor valor F1. Por motivos de cómputo, sólo se evaluarán hasta 10 árboles

#### 4.5.1. Conjunto con sobremuestreo

In [27]:
# Variables para guardar la mejor f1 con el número de de árboles
best_f1_forest_up = 0
best_estimator = 0

# Ciclo para evaluar el modelo en diferentes números de árboles
for est in range(1,11):
    model_forest_up = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_forest_up.fit(features_upsampled, target_upsampled)
    predictions_valid_forest = model_forest_up.predict(features_valid)
    f1 = f1_score(target_valid, predictions_valid_forest)
    if f1 > best_f1_forest_up:
        best_estimator = est
        best_f1_forest_up = f1

print(f'F1 del mejor modelo en el conjunto de validación (n_estimators={best_estimator}): {best_f1_forest_up}')

F1 del mejor modelo en el conjunto de validación (n_estimators=9): 0.5891016200294551


Valor AUC-ROC

In [28]:
# Probabilidades de clase
probabilities_valid = model_forest_up.predict_proba(features_valid)

# Probabilidades de clase positiva
probabilities_one_valid = probabilities_valid[:,1]

# valor AUC-ROC
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC_ROC:', auc_roc)

AUC_ROC: 0.8247518026765419


#### 4.5.2. Conjunto con submuestreo

In [29]:
# Variables para guardar la mejor f1 con el número de de árboles
best_f1_forest_down = 0
best_estimator = 0

# Ciclo para evaluar el modelo en diferentes números de árboles
for est in range(1,11):
    model_forest_down = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_forest_down.fit(features_downsampled, target_downsampled)
    predictions_valid_forest = model_forest_down.predict(features_valid)
    f1 = f1_score(target_valid, predictions_valid_forest)
    if f1 > best_f1_forest_down:
        best_estimator = est
        best_f1_forest_down = f1

print(f'F1 del mejor modelo en el conjunto de validación (n_estimators={best_estimator}): {best_f1_forest_down}')

F1 del mejor modelo en el conjunto de validación (n_estimators=10): 0.5392953929539295


Valor AUC-ROC

In [30]:
# Probabilidades de clase
probabilities_valid = model_forest_down.predict_proba(features_valid)

# Probabilidades de clase positiva
probabilities_one_valid = probabilities_valid[:,1]

# valor AUC-ROC
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC_ROC:', auc_roc)

AUC_ROC: 0.800109891764139


Conclusiones:

El valor F1 mejoró en ambos conjuntos, así como el valor mejoró el valor AUC-ROC. Pero, aún no se logra el valor F1 esperado de 0.59

### 4.6. Regresión Logística

Se entrena el modelo con `solver=liblinear`

#### 4.6.1. Conjunto con sobremuestro

In [31]:
model_regression_up = LogisticRegression(random_state=12345, solver='liblinear')
model_regression_up.fit(features_upsampled, target_upsampled)
predictions_valid_regression = model_regression_up.predict(features_valid)
f1_regression_up = f1_score(target_valid, predictions_valid_regression)
print('Valor F1:', f1_regression_up)

Valor F1: 0.42993024730500945


Valor AUC-ROC

In [32]:
# Probabilidades de clase
probabilities_valid = model_regression_up.predict_proba(features_valid)

# Probabilidades de clase positiva
probabilities_one_valid = probabilities_valid[:,1]

# valor AUC-ROC
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC_ROC:', auc_roc)

AUC_ROC: 0.7747651628054012


#### 4.6.2. Conjunto con submuestro

In [33]:
model_regression_down = LogisticRegression(random_state=12345, solver='liblinear')
model_regression_down.fit(features_downsampled, target_downsampled)
predictions_valid_regression = model_regression_down.predict(features_valid)
f1_regression_down = f1_score(target_valid, predictions_valid_regression)
print('Valor F1:', f1_regression_down)

Valor F1: 0.4920212765957447


Valor AUC-ROC

In [34]:
# Probabilidades de clase
probabilities_valid = model_regression_down.predict_proba(features_valid)

# Probabilidades de clase positiva
probabilities_one_valid = probabilities_valid[:,1]

# valor AUC-ROC
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('AUC_ROC:', auc_roc)

AUC_ROC: 0.7743794126059401


Conclusiones:

No se logra el valor F1 deseado

### 4.7. Conclusiones

Se evaluarón diferentes valores de `'repeat'` para el sobremuestreo y de `'fraction'` para el submuestreo, encontrando los mejores valores que se muestran debajo:

In [35]:
print(f'repeat = {repeat}, fraction = {fraction}')

print(f'Árbol de decisión: normal = {best_f1_tree}, sobremuestreo = {best_f1_tree_up}, submuestreo = {best_f1_tree_down}')
print(f'Bosque aleatorio: normal = {best_f1_forest}, sobremuestreo = {best_f1_forest_up}, submuestreo = {best_f1_forest_down}')
print(f'Regresión logística: normal = {f1_regression}, sobremuestreo = {f1_regression_up}, submuestreo = {f1_regression_down}')

repeat = 9, fraction = 0.4
Árbol de decisión: normal = 0.557427258805513, sobremuestreo = 0.5048715677590788, submuestreo = 0.577127659574468
Bosque aleatorio: normal = 0.523961661341853, sobremuestreo = 0.5891016200294551, submuestreo = 0.5392953929539295
Regresión logística: normal = 0.3004115226337448, sobremuestreo = 0.42993024730500945, submuestreo = 0.4920212765957447


El valor más cercano de 0.59 es con el modelo de bosque aleatorio con el conjunto de sobremuestreo con un valor de F1 = 0.589, por lo tanto este es el modelo que se ocupará para el conjunto de prueba.

Como se puede ver en el apartado 4.5.1. El `n_estimators` con el que dió dicho resultado es 9.

Además también es el modelo con el mejor valor AUC-ROC, el cuál es de 0.824

## 5. Prueba final

Para poder evaluar el conjunto de prueba, se van a estandarizar las variables usando `StandardScaler()`. Esto debido a que en los temas anteriores no se había realizado este proceso con el conjunto de prueba.

In [36]:
# Escalado del conjunto de prueba
scaler_test = StandardScaler()
scaler_test.fit(features_test[numeric])
features_test[numeric] = scaler_test.transform(features_test[numeric])

/tmp/ipykernel_27/2157777557.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_test[numeric] = scaler_test.transform(features_test[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [37]:
final_model = RandomForestClassifier(random_state=12345, n_estimators=9)
final_model.fit(features_upsampled, target_upsampled)
predictions_test = final_model.predict(features_test)
f1_test = f1_score(target_test, predictions_test)
print('Valor F1 del conjunto de prueba:', f1_test)

Valor F1 del conjunto de prueba: 0.5310734463276837


Conclusión:

EL valor F1 del conjunto de prueba es de 0.513, el cuál es un poco por debajo de lo que se nos pide.